In [ ]:
import pandas as pd
import math
import numpy as np
import os
from matplotlib import pyplot

In [ ]:

## Run Percolator
def run_percolator(pin_file, out_target, out_decoy):
    os.system("percolator -i 10 --default-direction MainScore \
    --testFDR 0.01 --trainFDR 0.01 --results-psms "+out_target+" \
    --decoy-results-psms "+out_decoy+" -P random_ -Y -U "+pin_file)

## Based on the results from Percolator, it extracts top-scored PRSMs
## from the pXg result.
def select_top_scored_pxg_res(pxg_res_file, p_target_file, p_decoy_file):
    pxg_res = pd.read_table(pxg_res_file)
    p_target = pd.read_table(p_target_file)
    p_decoy = pd.read_table(p_decoy_file)
    
    ## remove "random_" mark in decoy
    p_decoy = p_decoy.replace({'random_': ''}, regex=True)
    
    pxg_res['percolator_score'] = -100

    ## ID hash
    gid_dict = {}
    score_dict = {}
    for idx in p_target.index:
        gid_dict[p_target['PSMId'][idx]] = p_target['proteinIds'][idx]
        score_dict[p_target['PSMId'][idx]] = p_target['score'][idx]
    
    for idx in p_decoy.index:
        gid_dict[p_decoy['PSMId'][idx]] = p_decoy['proteinIds'][idx]
        score_dict[p_decoy['PSMId'][idx]] = p_decoy['score'][idx]

    # SpecID
    # GenomicID
    for idx in pxg_res.index:
        spec_id = pxg_res['SpecID'][idx]
        genomic_id = pxg_res['GenomicID'][idx]
        if spec_id in gid_dict:
            if int(gid_dict[spec_id]) == int(genomic_id):
                pxg_res['percolator_score'][idx] = score_dict[spec_id]
    
    
    return pxg_res[pxg_res['percolator_score'] != -100]

## Estimate global FDR
def run_global_fdr(df, target_fdr, desc = "NONE"):
    
    target_cnt = 0
    decoy_cnt = 0
    score_threshold = 100
    
    x_max = max(df['percolator_score'])
    x_min = min(df['percolator_score'])
    
    bins = np.linspace(x_min, x_max, 100)
    pyplot.title(desc)
    pyplot.hist(df[df['Label']==1]['percolator_score'], bins, alpha=0.5, label='target')
    pyplot.hist(df[df['Label']==-1]['percolator_score'], bins, alpha=0.5, label='decoy')
    pyplot.legend(loc='upper right')
    pyplot.show()
    
    ## Pass stat.
    estimated_fdr = 0
    estimated_target = 0
    estimated_decoy = 0
    for idx in df.index:
        label = df['Label'][idx]
        
        if label == 1:
            target_cnt += 1
        elif label == -1:
            decoy_cnt += 1
        
        if target_cnt != 0:
            fdr = decoy_cnt/target_cnt
            
            # pass target FDR?
            if fdr < target_fdr:
                score_threshold = df['percolator_score'][idx]
                estimated_fdr = fdr
                estimated_target = target_cnt
                estimated_decoy = decoy_cnt
    
    print("FDR at "+str(estimated_fdr))
    print("# of targets: "+ str(estimated_target))
    print("# of decoys: "+ str(estimated_decoy))
    
    pass_df = df[(df['Label'] == 1) & (df['percolator_score'] >= score_threshold)]
    
    return pass_df

## Estimate separate FDR (canonical and noncanonical)
def run_separate_fdr(filter_df, target_fdr):
    filter_df = filter_df.sort_values(by=['percolator_score'], ascending=False)
    
    ## Separte canonical and noncanonical PSMs
    canonical_list = filter_df[filter_df['IsCanonical'] == True]
    noncanonical_list = filter_df[filter_df['IsCanonical'] == False]
    
    return (run_global_fdr(canonical_list, target_fdr, "Canonical PSMs"), run_global_fdr(noncanonical_list, target_fdr, "Noncanonical PSMs"))
    

In [ ]:

if __name__ == '__main__':
    
    run_percolator("toy.pXg.pin", "toy.target.psm", "toy.decoy.psm")
    
    ## select top-scored PRSMs in the pXg result file.
    df = select_top_scored_pxg_res("toy.pXg", "toy.target.psm", "toy.decoy.psm")
    
    ## c_df and nc_df contain target PSMs passing a given FDR threshold
    ## in canonical and noncanonical, respectively
    c_df, nc_df = run_separate_fdr(df, 0.05)
    


In [ ]:
len(nc_df)